In [1]:
import glob
import os
import pandas as pd
import pandas_profiling as pdp

In [2]:
# 指定したファイルパスのcsvファイルを全部読み込む関数
def read_csv_all(target_path = '../input/'):
    for file in glob.glob(target_path+'*.csv'):
        raw_name = 'raw_' + file.replace(target_path,'').replace('.csv','')
        exec('{} = pd.read_csv(\'{}\')'.format(raw_name, file), globals())
        print('read {} as {}'.format(file, raw_name))

In [3]:
read_csv_all()

read ../input/new_merchant_transactions.csv as raw_new_merchant_transactions
read ../input/test.csv as raw_test
read ../input/merchants.csv as raw_merchants


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/hojokenta/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-2918fc16d579>", line 1, in <module>
    read_csv_all()
  File "<ipython-input-2-9946e0ab26fd>", line 5, in read_csv_all
    exec('{} = pd.read_csv(\'{}\')'.format(raw_name, file), globals())
  File "<string>", line 1, in <module>
  File "/Users/hojokenta/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/pandas/io/parsers.py", line 697, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/Users/hojokenta/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/pandas/io/parsers.py", line 430, in _read
    data = parser.read(nrows)
  File "/Users/hojokenta/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/pandas/io/parsers.py", line 1149, in read
    df = DataFrame(col_dict, columns=columns, index=inde

KeyboardInterrupt: 

In [5]:
pdp.ProfileReport(raw_train)

Number of variables,6
Number of observations,201917
Total Missing (%),0.0%
Total size in memory,9.2 MiB
Average record size in memory,48.0 B
Numeric,3
Categorical,1
Boolean,1
Date,0
Text (Unique),1
Rejected,0


### memo
 * targetとfeatureが全然相関してないから、このままだと全く当たらなそう

In [6]:
pdp.ProfileReport(raw_test)

Number of variables,5
Number of observations,123623
Total Missing (%),0.0%
Total size in memory,4.7 MiB
Average record size in memory,40.0 B
Numeric,2
Categorical,1
Boolean,1
Date,0
Text (Unique),1
Rejected,0


In [7]:
# 同じcard_idが複数名いるか確認
print('number of row in raw_train is {}. number of unique card_id is {}'.format(raw_train.shape[0], len(raw_train['card_id'].unique())))
print('number of row in raw_test is {}. number of unique card_id is {}'.format(raw_test.shape[0], len(raw_test['card_id'].unique())))

number of row in raw_train is 201917. number of unique card_id is 201917
number of row in raw_test is 123623. number of unique card_id is 123623


### memo
 * trainとtestの比率は5:3くらい
 * train、testともにcard_idに重複はない

In [8]:
# trainとtestにcard_idの重複があるか確認
print(sum(raw_train['card_id'].isin(raw_test['card_id'])))
print(sum(raw_test['card_id'].isin(raw_train['card_id'])))

0
0


### memo
 * trainとtestで重複しているidは存在しない

In [9]:
raw_new_merchant_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [10]:
# month_lag（基準日との差をちょっと調べる
raw_new_merchant_transactions[raw_new_merchant_transactions['month_lag'] == 1][['month_lag', 'purchase_date']].head()

,month_lag,purchase_date
0,1,2018-03-11 14:57:36
1,1,2018-03-19 18:53:37
3,1,2018-03-07 09:43:21
4,1,2018-03-22 21:07:53
6,1,2018-03-28 19:50:19


In [11]:
raw_new_merchant_transactions[raw_new_merchant_transactions['month_lag'] == 2][['month_lag', 'purchase_date']].head()

,month_lag,purchase_date
2,2,2018-04-26 14:08:44
5,2,2018-04-02 12:53:28
7,2,2018-04-05 08:06:52
8,2,2018-04-07 18:37:40
13,2,2018-04-14 09:27:45


In [12]:
raw_new_merchant_transactions[raw_new_merchant_transactions['month_lag'] == 3][['month_lag', 'purchase_date']].head()

,month_lag,purchase_date


In [13]:
raw_new_merchant_transactions['month_lag'].unique()

array([1, 2])

### memo
 * month_lagの値は1か2のいずれか
 * lag1が3月、lag2が4月を指してるっぽい

In [14]:
pdp.ProfileReport(raw_new_merchant_transactions)

Number of variables,14
Number of observations,1963031
Total Missing (%),0.7%
Total size in memory,209.7 MiB
Average record size in memory,112.0 B
Numeric,7
Categorical,5
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [15]:
# authorized_flag = 'N'のレコードを確認する
raw_new_merchant_transactions[raw_new_merchant_transactions['authorized_flag'] == 'N'].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id


### memo
 * authorized = 'N'は存在しない

In [16]:
# merchant_id is nullのレコードを確認する
raw_new_merchant_transactions[raw_new_merchant_transactions['merchant_id'].isnull()].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
51,Y,C_ID_8c9ba46f19,17,N,0,A,2,NaN,2,1.597228,2017-12-19 08:07:34,4.0,22,20
99,Y,C_ID_0a6ceeffcc,143,N,0,A,884,NaN,1,-0.596673,2017-11-24 00:00:00,5.0,5,27
154,Y,C_ID_d15cd87e37,73,N,0,A,108,NaN,1,-0.296112,2018-03-28 14:22:31,3.0,19,34
265,Y,C_ID_faacef6d77,69,N,0,A,307,NaN,1,-0.671775,2018-02-01 09:42:32,1.0,9,19
304,Y,C_ID_e4d60467c2,48,N,1,B,703,NaN,1,-0.680791,2018-03-25 10:37:28,1.0,9,29


### memo
 * 特に変なレコードが含まれているわけではないので一旦無視

In [17]:
# installmentsの数値ごとのレコード数を調査
raw_new_merchant_transactions.groupby('installments').size()

installments
-1       55922
 0      922244
 1      836178
 2       54729
 3       44750
 4       14815
 5        9296
 6       10389
 7         863
 8        1555
 9         478
 10       8899
 11         61
 12       2850
 999         2
dtype: int64

### memo
 * -1は返品、999は例外処理的ななにかかな？
 * 偶数が多いのはセット商品的なやつかな？

In [18]:
# installments = -1を確認
raw_new_merchant_transactions[raw_new_merchant_transactions['installments'] == -1].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
10,Y,C_ID_ef55cf8d4b,69,N,-1,NaN,45,M_ID_3ffd43b4cd,1,4.452265,2018-03-31 09:55:40,1.0,9,18
189,Y,C_ID_fd55871fd8,187,N,-1,NaN,367,M_ID_2a7d376053,1,-0.329366,2018-02-10 09:33:55,1.0,15,16
270,Y,C_ID_6bf003900f,69,N,-1,NaN,884,M_ID_1b3e5a9b05,1,-0.596673,2018-03-02 18:00:10,1.0,9,27
306,Y,C_ID_e4d60467c2,69,N,-1,NaN,274,M_ID_2da56d089d,1,-0.596643,2018-03-22 13:49:08,1.0,9,36
402,Y,C_ID_cb2269c52d,213,N,-1,NaN,34,M_ID_6972fc7624,2,-0.416325,2018-04-26 14:36:11,1.0,9,38


In [19]:
# 先頭行と同じcard_idとmerchant_idを抜き出してみる
raw_new_merchant_transactions[(raw_new_merchant_transactions['card_id'] == 'C_ID_ef55cf8d4b') & (raw_new_merchant_transactions['merchant_id'] == 'M_ID_3ffd43b4cd')].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
10,Y,C_ID_ef55cf8d4b,69,N,-1,NaN,45,M_ID_3ffd43b4cd,1,4.452265,2018-03-31 09:55:40,1.0,9,18


In [20]:
# installments = 999を確認
raw_new_merchant_transactions[raw_new_merchant_transactions['installments'] == 999]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
231851,Y,C_ID_8b3e96516a,174,N,999,C,840,M_ID_fcb061b2bf,1,-0.027138,2017-04-24 20:04:23,1.0,9,20
631387,Y,C_ID_726ae3c2bc,174,N,999,C,840,M_ID_fcb061b2bf,2,1.032186,2017-12-23 23:57:41,1.0,9,20


### memo
* installments=-1の意味はよくわからない。ひつまず入れておく
* installments =999もよくわからない。ただ、installmentsは連続値として扱いたいし、異常値なので、外しておくのが良さそう

In [21]:
# purchase_amountの最大値や大きい値を取っているcard_idがtrainに入っているか調査
raw_new_merchant_transactions.sort_values('purchase_amount', ascending = False).head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1676431,Y,C_ID_ca850ecf25,69,N,9,C,210,M_ID_8c654cb52e,2,263.157498,2018-04-06 15:52:38,1.0,9,35
228412,Y,C_ID_a289e7edd4,-1,Y,10,C,210,M_ID_d3efd95cc7,2,139.053747,2018-04-19 11:51:18,NaN,-1,35
893769,Y,C_ID_765f1b0537,69,N,-1,NaN,637,M_ID_f7227b1aa8,1,97.009567,2018-03-20 10:35:53,1.0,9,37
1815869,Y,C_ID_cc5b7d5b8a,69,N,5,C,210,M_ID_63154de5bf,1,76.682904,2018-01-05 17:35:21,1.0,9,35
866037,Y,C_ID_06dc178b0c,140,N,-1,NaN,222,M_ID_0f387af072,2,74.385651,2018-04-26 08:49:17,1.0,9,21


In [33]:
purchase_amount_top5 = raw_new_merchant_transactions.sort_values('purchase_amount', ascending = False).iloc[:10, :]
purchase_amount_top5

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1676431,Y,C_ID_ca850ecf25,69,N,9,C,210,M_ID_8c654cb52e,2,263.157498,2018-04-06 15:52:38,1.0,9,35
228412,Y,C_ID_a289e7edd4,-1,Y,10,C,210,M_ID_d3efd95cc7,2,139.053747,2018-04-19 11:51:18,NaN,-1,35
893769,Y,C_ID_765f1b0537,69,N,-1,NaN,637,M_ID_f7227b1aa8,1,97.009567,2018-03-20 10:35:53,1.0,9,37
1815869,Y,C_ID_cc5b7d5b8a,69,N,5,C,210,M_ID_63154de5bf,1,76.682904,2018-01-05 17:35:21,1.0,9,35
866037,Y,C_ID_06dc178b0c,140,N,-1,NaN,222,M_ID_0f387af072,2,74.385651,2018-04-26 08:49:17,1.0,9,21
250127,Y,C_ID_137b455c93,51,N,10,C,222,M_ID_0e2048155b,2,74.385651,2018-04-19 13:34:13,1.0,16,21
393414,Y,C_ID_48ff9c4776,143,N,10,C,333,M_ID_737b75be7c,1,74.385651,2018-03-06 14:11:59,5.0,5,10
1217334,Y,C_ID_ac666b220b,98,N,1,B,437,M_ID_0dbd2b61b4,1,72.452641,2017-12-13 15:13:36,1.0,9,15
124707,Y,C_ID_72613043cf,158,N,1,B,793,M_ID_faafc57543,2,68.224781,2018-04-14 13:11:52,1.0,15,38
155766,Y,C_ID_69fe79e230,211,N,3,C,333,M_ID_f8dae0d0d6,1,66.872395,2018-01-09 11:45:29,3.0,19,10


In [34]:
# purchase_amountが異常に大きい値を取っているレコードに関して、targetの値を確認してみる
raw_train[raw_train['card_id'].isin(purchase_amount_top5['card_id'])]

,first_active_month,card_id,feature_1,feature_2,feature_3,target
10575,2016-09,C_ID_72613043cf,1,2,0,-2.998747
24212,2017-09,C_ID_cc5b7d5b8a,1,2,0,-2.845817
81245,2017-08,C_ID_06dc178b0c,1,1,0,-3.476014
106666,2017-10,C_ID_69fe79e230,1,3,0,-3.268655
130469,2017-09,C_ID_137b455c93,1,1,0,-1.407583
147092,2017-08,C_ID_ac666b220b,1,1,0,-3.508950
153747,2017-11,C_ID_48ff9c4776,1,1,0,-3.463238


In [35]:
# testにも存在しているか確認
raw_test[raw_test['card_id'].isin(purchase_amount_top5['card_id'])]

,first_active_month,card_id,feature_1,feature_2,feature_3
37708,2017-01,C_ID_765f1b0537,1,1,0
52799,2017-01,C_ID_ca850ecf25,1,2,0
79747,2017-09,C_ID_a289e7edd4,1,1,0


### memo
 * purchase_amountがかなり大きい値を取っていると、ターゲットはそこそこ大きい負の値を取りそう。
 * testデータにもpurchase_amoutがマイナスのものをいくつかあるので、これらのデータがそれっぽい予測値を返しているかは確認しておきたい。

In [37]:
# subsector_id=-1のcard_idがtrainに入っているか調査
raw_new_merchant_transactions[raw_new_merchant_transactions['subsector_id'] == -1].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
47116,Y,C_ID_63c8bf7aa3,-1,Y,3,C,-1,M_ID_49d4a8510e,1,-0.357105,2017-08-25 12:05:43,NaN,-1,-1
366938,Y,C_ID_f102804a32,-1,Y,12,C,-1,M_ID_49d4a8510e,2,0.301823,2017-07-20 18:33:06,NaN,-1,-1
471345,Y,C_ID_dd9336b377,-1,Y,1,B,-1,M_ID_49d4a8510e,1,-0.721363,2017-09-29 00:47:23,NaN,-1,-1
545828,Y,C_ID_0be2d87124,-1,Y,1,B,-1,M_ID_49d4a8510e,2,-0.694315,2017-09-17 12:49:03,NaN,-1,-1
583369,Y,C_ID_b2c9f4e318,-1,Y,1,B,-1,M_ID_49d4a8510e,2,-0.300620,2017-08-03 10:26:38,NaN,-1,-1


In [38]:
# subsector_idのidごとの行数をカウント
raw_new_merchant_transactions.groupby('subsector_id').size()

subsector_id
-1         20
 1      66913
 2      15851
 3       6237
 4       4302
 5       1316
 7      54145
 8      14979
 9       7590
 10      9849
 11       560
 12     21351
 13      2419
 14      1488
 15     29772
 16    119297
 17     23233
 18     42079
 19    200528
 20     38433
 21     74176
 22     16460
 23      1423
 24      1907
 25     23805
 26      2221
 27    194977
 28         8
 29    138039
 30     31443
 31     15000
 32     16673
 33    172585
 34    183700
 35      5233
 36     36621
 37    340053
 38     13057
 39     21487
 40       953
 41     12848
dtype: int64

In [41]:
# subsector_id=-1のcard_idがtrain,testそれぞれに含まれているか確認する
subsector_id_minus1 = raw_new_merchant_transactions[raw_new_merchant_transactions['subsector_id'] == -1]
raw_train[raw_train['card_id'].isin(subsector_id_minus1['card_id'])]

,first_active_month,card_id,feature_1,feature_2,feature_3,target
25933,2016-12,C_ID_f4cd0e26ab,5,1,1,-0.243861
35068,2017-05,C_ID_55efb85ac2,1,1,0,0.503830
65800,2017-06,C_ID_dd9336b377,4,3,0,-0.017003
70112,2017-05,C_ID_63c8bf7aa3,1,3,0,-2.070915
73391,2017-05,C_ID_31d2bbb6aa,1,1,0,2.124241
91894,2016-10,C_ID_f102804a32,4,2,0,-0.472232
100414,2017-01,C_ID_0eb273835a,5,1,1,-0.598439
145042,2016-04,C_ID_fffa99d1da,4,2,0,-1.495226
154470,2016-10,C_ID_5a2803fe55,5,1,1,1.514334
164144,2016-01,C_ID_49909d86a9,2,2,0,-33.219281


In [42]:
subsector_id_minus1

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
47116,Y,C_ID_63c8bf7aa3,-1,Y,3,C,-1,M_ID_49d4a8510e,1,-0.357105,2017-08-25 12:05:43,NaN,-1,-1
366938,Y,C_ID_f102804a32,-1,Y,12,C,-1,M_ID_49d4a8510e,2,0.301823,2017-07-20 18:33:06,NaN,-1,-1
471345,Y,C_ID_dd9336b377,-1,Y,1,B,-1,M_ID_49d4a8510e,1,-0.721363,2017-09-29 00:47:23,NaN,-1,-1
545828,Y,C_ID_0be2d87124,-1,Y,1,B,-1,M_ID_49d4a8510e,2,-0.694315,2017-09-17 12:49:03,NaN,-1,-1
583369,Y,C_ID_b2c9f4e318,-1,Y,1,B,-1,M_ID_49d4a8510e,2,-0.300620,2017-08-03 10:26:38,NaN,-1,-1
657058,Y,C_ID_0eb273835a,-1,Y,1,B,-1,M_ID_49d4a8510e,1,-0.556071,2017-07-11 16:27:08,NaN,-1,-1
705425,Y,C_ID_6ba7520bc4,-1,Y,1,B,-1,M_ID_49d4a8510e,2,-0.694315,2017-08-03 20:59:59,NaN,-1,-1
760952,Y,C_ID_1a67f0cf03,-1,Y,10,C,-1,M_ID_49d4a8510e,1,-0.412267,2017-07-17 08:05:37,NaN,-1,-1
1070360,Y,C_ID_55efb85ac2,-1,Y,10,C,-1,M_ID_49d4a8510e,1,0.277600,2017-07-05 23:23:41,NaN,-1,-1
1111986,Y,C_ID_fffa99d1da,-1,Y,1,B,-1,M_ID_49d4a8510e,1,0.300440,2017-08-17 20:46:00,NaN,-1,-1


### memo
 * subsector_id = -1の中にtargetの値が異常値（-33.219281）のものが存在。
 * 他のcard_idと比較して対しておかしいと思うところはないため、他のテーブルで変なところはないか要調査。card_id = C_ID_49909d86a9

In [44]:
# 流れでcard_id = C_ID_49909d86a9を調査
raw_new_merchant_transactions[raw_new_merchant_transactions['card_id'] == 'C_ID_49909d86a9']

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1793091,Y,C_ID_49909d86a9,-1,Y,1,B,574,M_ID_b43e83f5ee,2,-0.426573,2017-09-05 18:53:26,NaN,-1,22
1793092,Y,C_ID_49909d86a9,-1,Y,12,C,-1,M_ID_49d4a8510e,1,0.658492,2017-08-08 21:12:50,NaN,-1,-1


In [51]:
# 流れでmerchant_category_id=574を調査
raw_new_merchant_transactions[raw_new_merchant_transactions['merchant_category_id']==574].shape

(7726, 14)

### memo
 * targetの値がめっちゃちっちゃいcard_idで特に変だと感じるところはない
 * 単純に除外しても良いかも
 * もしくは、他のテーブルで調査してみる

In [52]:
pdp.ProfileReport(raw_merchants)

Number of variables,22
Number of observations,334696
Total Missing (%),0.2%
Total size in memory,56.2 MiB
Average record size in memory,176.0 B
Numeric,13
Categorical,5
Boolean,0
Date,0
Text (Unique),0
Rejected,4


### memo
 * avg_purchases_lag12とavg_purchases_lag6とavg_purchases_lag3は相関が高いので、どれか1つだけ使えば良いかも
 * avg_sales_lag☓☓も同様
 * numerical_1と2も同様